### Library and modules

In [1]:
import numpy as np, pandas as pd, geopandas as gpd, sklearn.metrics as metrics
import geemap, ee , rasterio, sankee, warnings, pickle
from rasterio.plot import show
from geopandas import GeoDataFrame
from osgeo import gdal, osr, gdalconst
import os, glob, subprocess
import rasterio
import matplotlib as mpl
import matplotlib.pyplot as plt, plotly.express as px
from geemap import ml
from sklearn import ensemble
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from shapely.geometry import Polygon, Point
from sklearn.metrics import confusion_matrix, cohen_kappa_score
warnings.filterwarnings('ignore')

### Earth Engine log in

In [3]:
ee.Authenticate()

Enter verification code:  4/1AWgavdd541aJM5shL4CkFLN5Imue6HbVJCwBUUq9Dj3ospn9I6RxnNjOH8E



Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
######################## subset L8 collection   ########################################
##################################################################################
def getFactorImg(factorNames, image):
    factorList = image.toDictionary().select(factorNames).values()
    return ee.Image.constant(factorList)

######################## pandas normalise   ########################################
##################################################################################
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

######################## L8 cloud mask   ########################################
##################################################################################
def getQABits(image, start, end, mask):
    pattern = 0
    for i in range(start,end-1):
        pattern += 2**i
    return image.select([0], [mask]).bitwiseAnd(pattern).rightShift(start)


def maskQuality(image):
    QA = image.select('QA_PIXEL')
    shadowMask = getQABits(QA,3,3,'cloud_shadow')
    cloudMask = getQABits(QA,5,5,'cloud')
    cirrusMask = getQABits(QA,9,9,'cirrus_detected')
    scaleImg = getFactorImg(['REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'], image)
    offsetImg = getFactorImg(['REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'], image)
    scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)
    
    return (image.addBands(scaled, None, True)
            .updateMask(cirrusMask)
            # .updateMask(cloudMask)
            # .updateMask(shadowMask)
            )



def prepSrL8(image):
    dilateMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 0)).eq(0)
    cirrusMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    cloudMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 3)).eq(0)
    shadowMask =  image.select('QA_PIXEL').bitwiseAnd(int('11111', 4)).eq(0)
    snowMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 5)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    scaleImg = getFactorImg(['REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'], image)
    offsetImg = getFactorImg(['REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'], image)
    scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)

    return (image.addBands(scaled, None, True)
                 .updateMask(cirrusMask)
                 # .updateMask(dilateMask)
                 # .updateMask(cloudMask)
                 # .updateMask(shadowMask)
                 # .updateMask(snowMask)
                 #.updateMask(saturationMask)
           )

######################## L8 scale factors   ####################################
##################################################################################
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

######################## convert L8 to SST   ####################################
##################################################################################
def sst(img):
    thermalBands = img.select('ST_TRAD')
    ln = k2_im.divide((k1_im.divide(thermalBands).add(1).log())).multiply(0.001).clip(site).rename("SST")
    #.convolve(kern)
    return img.addBands(ln)

######################## scale image   ####################################
##################################################################################
def scale(image):
    return ee.Image(image).multiply(0.001)

def addNDAVI(image):
    ndvi = image.normalizedDifference(['SR_B1', 'SR_B5']).rename('NDAVI')
    return image.addBands(ndvi)

def addNDWI(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI')
    return image.addBands(ndvi)

In [13]:
hab = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\BOSS\\Cleaned\\BOSS_hab.shp'))

sites = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\WGS84\\ICOAST_sites.shp'))

kal = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\Sites\\Kalbarri.shp'))

mask = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\Lidar mask\\Lidar_mask.shp'))

lidar_url = 'gs://cog-bucket/lidar_cog.tif'
lidar = geemap.load_GeoTIFF(lidar_url).rename('lidar')
hab = hab.clip(mask)

AttributeError: 'FeatureCollection' object has no attribute 'clip'

### L8 collection

In [5]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.centerObject(sites.geometry(), 5)
Map.addLayer(mask,  {}, 'Sites')
Map

Map(center=[-26.38096118269607, 113.62250401413422], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
start_date = '2021-02-01'
end_date = '2021-12-30'

L8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(mask)
    .filterMetadata('CLOUD_COVER', 'less_than', 5)
    .filterDate(start_date, end_date)
    #.filter("IMAGE_QUALITY > 7")
    .map(addNDAVI)
    .map(addNDWI)
    .map(prepSrL8)
    .median()
    .clip(mask)
    #.addBands(lidar.rename('lidar'))
)

In [14]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.centerObject(sites.geometry(), 5)
#Map.add_basemap('HYBRID')
Map.addLayer(L8, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'max': 0.05, 'min' : 0}, 'TOA composite')
Map.addLayer(sites,  {}, 'Sites')
Map.addLayer(hab,  {}, 'boss')
Map

Map(center=[-26.38096118269607, 113.62250401413422], controls=(WidgetControl(options=['position', 'transparent…

In [152]:
start_date = '2013-01-01'
end_date = '2013-12-30'

L8_13 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(sites)
    .filterMetadata('CLOUD_COVER', 'less_than', 5)
    .filterDate(start_date, end_date)
    #.filter("IMAGE_QUALITY > 7")
    .map(prepSrL8)
    .map(addNDAVI)
    .map(addNDWI)
    .median()
    .clip(sites)
)


In [148]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.centerObject(sites.geometry(), 5)
Map.add_basemap('HYBRID')
Map.addLayer(L8_13, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'max': 0.1, 'min' : 0}, 'TOA composite')
Map.addLayer(sites,  {}, 'Sites')
Map

Map(center=[-26.38096118269607, 113.62250401413422], controls=(WidgetControl(options=['position', 'transparent…

In [153]:
start_date = '2008-02-01'
end_date = '2008-06-30'

L5 = (
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterBounds(sites)
    .filterMetadata('CLOUD_COVER', 'less_than', 5)
    .filter("IMAGE_QUALITY == 9")
    .filterDate(start_date, end_date)
    .map(prepSrL8)
    .median()
    .clip(sites)
)

In [150]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.centerObject(sites.geometry(), 5)
Map.add_basemap('HYBRID')
Map.addLayer(L5, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'max': 0.1, 'min' : 0}, 'TOA composite')
Map.addLayer(sites,  {}, 'Sites')
Map

Map(center=[-26.38096118269607, 113.62250401413422], controls=(WidgetControl(options=['position', 'transparent…

In [8]:
proj = ee.Projection('EPSG:4326')
ndavi_dat = geemap.extract_values_to_points(hab,
                                            L8.select('NDAVI'), 
                                            crs = proj, 
                                            scale = 30)

ndwi_dat = geemap.extract_values_to_points(hab,
                                            L8.select('NDWI'), 
                                            crs = proj, 
                                            scale = 30)

b1_dat = geemap.extract_values_to_points(hab,
                                         L8.select('SR_B1'), 
                                         crs = proj, 
                                         scale = 30)

gr_dat = geemap.extract_values_to_points(hab,
                                         L8.select('SR_B2'), 
                                         crs = proj, 
                                         scale = 30)

# lid_dat = geemap.extract_values_to_points(hab,
#                                          L8.select('lidar'), 
#                                          crs = proj, 
#                                          scale = 30)

ndavi_dat  = geemap.ee_to_pandas(ndavi_dat)
b1_dat  = geemap.ee_to_pandas(b1_dat)
ndwi_dat = geemap.ee_to_pandas(ndwi_dat)
gr_dat = geemap.ee_to_pandas(gr_dat)
#lid_dat = geemap.ee_to_pandas(lid_dat)

In [11]:
#ndwi_dat.info()
b1_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        408 non-null    object 
 1   broad_Ston  408 non-null    int64  
 2   image       408 non-null    object 
 3   Total_adj   408 non-null    int64  
 4   broad_Unkn  408 non-null    int64  
 5   visibility  408 non-null    object 
 6   broad_Spon  408 non-null    int64  
 7   latitude    408 non-null    float64
 8   fov_Limite  408 non-null    float64
 9   Cluster     408 non-null    object 
 10  broad_Seag  408 non-null    int64  
 11  mean_relie  408 non-null    float64
 12  sample      408 non-null    object 
 13  sd_relief   408 non-null    float64
 14  site        408 non-null    object 
 15  time_botto  408 non-null    object 
 16  broad_Macr  408 non-null    int64  
 17  location    408 non-null    object 
 18  broad_Cons  408 non-null    int64  
 19  fov_Open    408 non-null    f

In [213]:
ndavi_dat = ndavi_dat.rename(columns={'first': 'NDAVI'})
b1_dat = b1_dat.rename(columns={'first': 'SR_B1'})
ndwi_dat = ndwi_dat.rename(columns={'first': 'NDWI'})
gr_dat = gr_dat.rename(columns={'first': 'SR_B2'})
lid_dat = lid_dat.rename(columns={'first': 'lidar'})

train_df = ndavi_dat
train_df['SR_B1'] = b1_dat['SR_B1']
train_df['NDWI'] = ndwi_dat['NDWI']
train_df['SR_B2'] = gr_dat['SR_B2']
train_split = train_df.sample(frac=0.9,random_state=2)
train_split = train_split.dropna(subset=['NDAVI', 'SR_B1', 'NDWI', 'Cluster'])
test_split = train_df.drop(train_split.index)
test_split = test_split.dropna(subset=['NDAVI', 'SR_B1', 'NDWI', 'Cluster'])

feature_names = ['NDAVI', 'SR_B1', 'NDWI', 'SR_B2']
label = "Cluster"
X = train_split[feature_names]
y = train_split[label]

KeyError: 'SR_B1'

In [198]:
n_trees = 1000
rf = (
     ensemble.RandomForestClassifier(n_trees,
                                    max_features = 'sqrt',
                                    max_depth = 5,
                                    ccp_alpha=0.001,
                                    oob_score = True)
                                    .fit(X, y)
)

pred = rf.predict(test_split[feature_names])

In [200]:
confusion_matrix(test_split[label], pred)
cohen_kappa_score(test_split[label], pred)

0.39194915254237284

In [201]:
trees = ml.rf_to_strings(rf, 
                        feature_names,
                        processes=3,
                        output_mode = 'CLASSIFICATION')
ee_classifier = ml.strings_to_classifier(trees)
classified = L8.select(feature_names).classify(ee_classifier)

In [202]:
ee_classifier = ml.strings_to_classifier(trees)
classified = L8.select(feature_names).classify(ee_classifier)

In [203]:
Map_pred = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map_pred.add_basemap('HYBRID')
vis_pred = {
    'min': 0,
    'max': 1,
    'palette': ['#9EE953', '#D2B866', '#3A570B'],
}

Map_pred.addLayer(classified, vis_pred  ,'classification',
)

Map_pred


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [170]:
Kal_L8 = L8.select(['SR_B2', 'SR_B3', 'SR_B4']).clip(kal).unmask()
task_Kal_L8 = ee.batch.Export.image.toCloudStorage(**{
    'image': Kal_L8 ,
    'description': 'L8_Kal',
    'fileNamePrefix' :'L8_Kal',
    'crs': 'EPSG:4326',
    'scale': 30,
    'region': kal.geometry(),
    'fileFormat': 'GeoTIFF',
    'bucket': 'cog-bucket',
    'maxPixels': 1e13,
    'skipEmptyTiles' : True,
    'formatOptions': {'cloudOptimized': True}
})
task_Kal_L8.start()
task_Kal_L8.status()

{'state': 'READY',
 'description': 'L8_Kal',
 'creation_timestamp_ms': 1671077929332,
 'update_timestamp_ms': 1671077929332,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'JMFZZWS3W7FZES2HIE6622N5',
 'name': 'projects/earthengine-legacy/operations/JMFZZWS3W7FZES2HIE6622N5'}

In [174]:
task_Kal_L8.status()

{'state': 'RUNNING',
 'description': 'L8_Kal',
 'creation_timestamp_ms': 1671077929332,
 'update_timestamp_ms': 1671077973951,
 'start_timestamp_ms': 1671077973873,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'JMFZZWS3W7FZES2HIE6622N5',
 'name': 'projects/earthengine-legacy/operations/JMFZZWS3W7FZES2HIE6622N5'}

In [176]:
Kal_L8_13 = L8_13.select(['SR_B2', 'SR_B3', 'SR_B4']).clip(kal).unmask()
task_Kal_L8_13 = ee.batch.Export.image.toCloudStorage(**{
    'image': Kal_L8_13 ,
    'description': 'L8_13_Kal',
    'fileNamePrefix' :'L8_13_Kal',
    'crs': 'EPSG:4326',
    'scale': 30,
    'region': kal.geometry(),
    'fileFormat': 'GeoTIFF',
    'bucket': 'cog-bucket',
    'maxPixels': 1e13,
    'skipEmptyTiles' : True,
    'formatOptions': {'cloudOptimized': True}
})
task_Kal_L8_13.start()
task_Kal_L8_13.status()

{'state': 'READY',
 'description': 'L8_13_Kal',
 'creation_timestamp_ms': 1671077990649,
 'update_timestamp_ms': 1671077990649,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '4I7OU4WXIBEZXPGMHTYDGT2Y',
 'name': 'projects/earthengine-legacy/operations/4I7OU4WXIBEZXPGMHTYDGT2Y'}

In [175]:
task_Kal_L8_13.status()

{'state': 'FAILED',
 'description': 'L8_13_Kal',
 'creation_timestamp_ms': 1671077816104,
 'update_timestamp_ms': 1671077832240,
 'start_timestamp_ms': 1671077831646,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'error_message': 'Exported bands must have compatible data types; found inconsistent types: Float64 and Float32.',
 'id': 'B2S2XCAUTCH2D464YJDHRM5H',
 'name': 'projects/earthengine-legacy/operations/B2S2XCAUTCH2D464YJDHRM5H'}

In [168]:
Kal_L5 = L5.select(['SR_B1', 'SR_B2', 'SR_B3']).clip(kal).unmask()
task_Kal_L5 = ee.batch.Export.image.toCloudStorage(**{
    'image': Kal_L5 ,
    'description': 'L5_Kal',
    'fileNamePrefix' :'L5_Kal',
    'crs': 'EPSG:4326',
    'scale': 30,
    'region': kal.geometry(),
    'fileFormat': 'GeoTIFF',
    'bucket': 'cog-bucket',
    'maxPixels': 1e13,
    'skipEmptyTiles' : True,
    'formatOptions': {'cloudOptimized': True}
})
task_Kal_L5.start()
task_Kal_L5.status()

{'state': 'READY',
 'description': 'L5_Kal',
 'creation_timestamp_ms': 1671077854539,
 'update_timestamp_ms': 1671077854539,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'S2DPUQ44G4MHXAJ4DNTFVXMT',
 'name': 'projects/earthengine-legacy/operations/S2DPUQ44G4MHXAJ4DNTFVXMT'}

In [177]:
task_Kal_L5.status()

{'state': 'COMPLETED',
 'description': 'L5_Kal',
 'creation_timestamp_ms': 1671077854539,
 'update_timestamp_ms': 1671077977120,
 'start_timestamp_ms': 1671077890869,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/cog-bucket/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 3.0443482398986816,
 'id': 'S2DPUQ44G4MHXAJ4DNTFVXMT',
 'name': 'projects/earthengine-legacy/operations/S2DPUQ44G4MHXAJ4DNTFVXMT'}